### Evaluating CPM with CEBaB
Our Causal Proxy Model (CPM) is for providing concept-based explanation for a blackbox model. We use newly developed CEBaB benchmark for comparing CPM with other concept-based explanation methods. This notebook evaluates CPM with CEBaB benchmark under different settings.

More importantly, we introduce new baselines for CPM as well. Formally, we evaluate the blackbox model with interchange intervention evaluation (which will be introduced in details below).

In this notebook, we can evaluate the following models:
- CPM: `BERT-base-uncased`
- CPM: `RoBERTa-base`
- CPM: `GPT2`
- CPM: `LSTM+GloVe`
- CPM: `Control`

and we can evaluate with the following conditions:
- 2-class
- 3-class
- 5-class

#### Imports and Libs

In [1]:
from libs import *
from modelings.modelings_bert import *
from modelings.modelings_roberta import *
from modelings.modelings_gpt2 import *
from modelings.modelings_lstm import *
"""
For evaluate, we use a single random seed, as
the models are trained with 5 different seeds
already.
"""
_ = random.seed(123)
_ = np.random.seed(123)
_ = torch.manual_seed(123)

#### Main evaluate script

In [2]:
results = {}
# rerunning the boxes below will only append stuffs to the results.

In [3]:
"""
The following blocks will run CEBaB benchmark in
all the combinations of the following conditions.
"""
grid = {
    "eval_split": ["test"],
    # dev,test
    "control": ["baseline-random"],
    # baseline-random,hdims
    "seed": [42,66,77],
    # 42, 66, 77
    "h_dim": [192],
    # 1,16,64,128,192
    "interchange_layer" : [10],
    # 0,2,4,6,8,10,12
    "class_num": [5],
    "k" : [0], 
    # 0,10,100,500,1000,3000,6000,9848,19684
    "alpha" : [0.0],
    # 0.0,1.0
    "beta" : [0.0],
    # 0.0,1.0
    "gemma" : [0.0],
    # 0.0,3.0
    "model_arch" : ["bert-base-uncased"],
    "lr" : ["8e-05"],
    "counterfactual_type" : ["approximate"]
    # approximate,true
}

keys, values = zip(*grid.items())
permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

device = 'cuda:0'
batch_size = 32

In [ ]:

for i in range(len(permutations_dicts)):
    
    eval_split=permutations_dicts[i]["eval_split"]
    seed=permutations_dicts[i]["seed"]
    class_num=permutations_dicts[i]["class_num"]
    alpha=permutations_dicts[i]["alpha"]
    beta=permutations_dicts[i]["beta"]
    gemma=permutations_dicts[i]["gemma"]
    h_dim=permutations_dicts[i]["h_dim"]
    dataset_type = f'{class_num}-way'
    control=permutations_dicts[i]["control"]
    model_arch=permutations_dicts[i]["model_arch"]
    k=permutations_dicts[i]["k"]
    interchange_layer=permutations_dicts[i]["interchange_layer"]
    lr=permutations_dicts[i]["lr"]
    counterfactual_type=permutations_dicts[i]["counterfactual_type"]
    
    if model_arch == "bert-base-uncased":
        model_path = "BERT"
        model_module = BERTForCEBaB
        explainer_module = CausalProxyModelForBERT
    elif model_arch == "roberta-base":
        model_path = "RoBERTa" 
        model_module = RoBERTaForCEBaB
        explainer_module = CausalProxyModelForRoBERTa
    elif model_arch == "gpt2":
        model_path = "gpt2"
        model_module = GPT2ForCEBaB
        explainer_module = CausalProxyModelForGPT2
    elif model_arch == "lstm":
        model_path = "lstm"
        model_module = LSTMForCEBaB
        explainer_module = CausalProxyModelForLSTM
    model_path += f"-{control}"
    grid_conditions=(
        ("eval_split", eval_split),
        ("control", control),
        ("seed", seed),
        ("h_dim", h_dim),
        ("interchange_layer", interchange_layer),
        ("class_num", class_num),
        ("k", k),
        ("alpha", alpha),
        ("beta", beta),
        ("gemma", gemma),
        ("model_arch", model_arch),
        ("lr", lr),
        ("counterfactual_type", counterfactual_type)
    )
    print("Running for this setting: ", grid_conditions)

    blackbox_model_path = f'CEBaB/{model_arch}.CEBaB.sa.'\
                          f'{class_num}-class.exclusive.seed_{seed}'
    cpm_model_path = f'../proxy_training_results/{model_path}/'\
                     f'cebab.alpha.{alpha}.beta.{beta}.gemma.{gemma}.'\
                     f'lr.{lr}.dim.{h_dim}.hightype.{model_arch}.'\
                     f'CEBaB.cls.dropout.0.1.enc.dropout.0.1.counter.type.'\
                     f'{counterfactual_type}.k.{k}.int.layer.{interchange_layer}.'\
                     f'seed_{seed}/'

    # load data from HF
    cebab = datasets.load_dataset(
        'CEBaB/CEBaB', use_auth_token=True,
        cache_dir="../train_cache/"
    )

    train, dev, test = preprocess_hf_dataset_inclusive(
        cebab, verbose=1, dataset_type=dataset_type
    )

    eval_dataset = dev if eval_split == 'dev' else test

    tf_model = model_module(
        blackbox_model_path, 
        device=device, 
        batch_size=batch_size
    )
    explainer = explainer_module(
        blackbox_model_path,
        cpm_model_path, 
        device=device, 
        batch_size=batch_size,
        intervention_h_dim=h_dim,
    )

    result_per_example, ATE, CEBaB_metrics, CEBaB_metrics_per_aspect_direction, \
    CEBaB_metrics_per_aspect, CaCE_per_aspect_direction, \
    ACaCE_per_aspect, performance_report = cebab_pipeline(
        tf_model, explainer, 
        train, eval_dataset,
        seed, k, dataset_type=dataset_type, 
        shorten_model_name=False, 
        train_setting="inclusive", 
        approximate=False if counterfactual_type == "true" else True
    )
    
    results[grid_conditions] = (
        result_per_example, ATE, CEBaB_metrics, CEBaB_metrics_per_aspect_direction, \
        CEBaB_metrics_per_aspect, CaCE_per_aspect_direction, \
        ACaCE_per_aspect, performance_report
    )

Running for this setting:  (('eval_split', 'test'), ('control', 'baseline-random'), ('seed', 42), ('h_dim', 192), ('interchange_layer', 10), ('class_num', 5), ('k', 0), ('alpha', 0.0), ('beta', 0.0), ('gemma', 0.0), ('model_arch', 'bert-base-uncased'), ('lr', '8e-05'), ('counterfactual_type', 'approximate'))


Using custom data configuration CEBaB--CEBaB-ccd674d249652bd4
Reusing dataset parquet (../train_cache/CEBaB___parquet/CEBaB--CEBaB-ccd674d249652bd4/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/5 [00:00<?, ?it/s]

Dropping no majority reviews: 16.6382% of train_exclusive dataset.
Dropping no majority reviews: 16.03% of train_inclusive dataset.
intervention_h_dim=192


../eval_pipeline/utils/data_utils.py:219: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs['intervention_type'][pairs[f'{aspect}_aspect_majority_base'] != pairs[f'{aspect}_aspect_majority_counterfactual']] = aspect
  0%|          | 0/124 [00:00<?, ?it/s]/dfs/user/wuzhengx/tool-chain/anaconda3/lib/python3.7/site-packages/transformers/modeling_utils.py:714: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning
100%|██████████| 124/124 [00:18<00:00,  6.62it/s]
/afs/cs.stanford.edu/u/wuzhengx/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no p

Running for this setting:  (('eval_split', 'test'), ('control', 'baseline-random'), ('seed', 66), ('h_dim', 192), ('interchange_layer', 10), ('class_num', 5), ('k', 0), ('alpha', 0.0), ('beta', 0.0), ('gemma', 0.0), ('model_arch', 'bert-base-uncased'), ('lr', '8e-05'), ('counterfactual_type', 'approximate'))


Using custom data configuration CEBaB--CEBaB-ccd674d249652bd4
Reusing dataset parquet (../train_cache/CEBaB___parquet/CEBaB--CEBaB-ccd674d249652bd4/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/5 [00:00<?, ?it/s]

Dropping no majority reviews: 16.6382% of train_exclusive dataset.
Dropping no majority reviews: 16.03% of train_inclusive dataset.


#### Show your results

In [21]:
important_keys = [
    "control", "seed", 
    "h_dim", "interchange_layer", 
    "class_num", "k", 
    "beta", "gemma", 
    "model_arch", "lr", "counterfactual_type"
]
values = []
for k, v in results.items():
    _values = []
    for ik in important_keys:
        _values.append(dict(k)[ik])
    _values.append(v[2]["ICaCE-L2"].iloc[0])
    _values.append(v[2]["ICaCE-cosine"].iloc[0])
    _values.append(v[2]["ICaCE-normdiff"].iloc[0])
    _values.append(v[-1].iloc[0][0])
    values.append(_values)
important_keys.extend(["ICaCE-L2", "ICaCE-cosine", "ICaCE-normdiff", "macro-f1"])
df = pd.DataFrame(values, columns=important_keys)
df.sort_values(by=['seed', 'interchange_layer', 'h_dim'], ascending=True)

,control,seed,h_dim,interchange_layer,class_num,k,beta,gemma,model_arch,lr,counterfactual_type,ICaCE-L2,ICaCE-cosine,ICaCE-normdiff,macro-f1
0,hdims,42,1,10,5,19684,1.0,3.0,bert-base-uncased,8e-05,true,0.4585,0.3766,0.2713,0.697301
1,hdims,42,16,10,5,19684,1.0,3.0,bert-base-uncased,8e-05,true,0.4403,0.3637,0.2583,0.684788
2,hdims,42,64,10,5,19684,1.0,3.0,bert-base-uncased,8e-05,true,0.4579,0.3786,0.2747,0.682398
3,hdims,42,128,10,5,19684,1.0,3.0,bert-base-uncased,8e-05,true,0.4433,0.3680,0.2571,0.695235
4,hdims,42,192,10,5,19684,1.0,3.0,bert-base-uncased,8e-05,true,0.4485,0.3571,0.2553,0.685358


#### Aggregating results in the way you like

In [15]:
df.groupby(['interchange_layer'], as_index=False).mean()

,interchange_layer,seed,h_dim,class_num,k,beta,gemma,ICaCE-L2,ICaCE-cosine,ICaCE-normdiff,macro-f1
0,2,61.666667,192.0,5.0,19684.0,1.0,3.0,0.461100,0.403400,0.273267,0.667550
1,4,61.666667,192.0,5.0,19684.0,1.0,3.0,0.462033,0.403367,0.269633,0.685080
2,6,61.666667,192.0,5.0,19684.0,1.0,3.0,0.452933,0.401867,0.260933,0.685926
3,8,61.666667,192.0,5.0,19684.0,1.0,3.0,0.461600,0.403833,0.271267,0.690230
4,10,61.666667,192.0,5.0,19684.0,1.0,3.0,0.446733,0.386767,0.258500,0.697420
5,12,61.666667,192.0,5.0,19684.0,1.0,3.0,0.470367,0.400267,0.281967,0.703554


In [16]:
df.groupby(['interchange_layer'], as_index=False).std()

,interchange_layer,seed,h_dim,class_num,k,beta,gemma,ICaCE-L2,ICaCE-cosine,ICaCE-normdiff,macro-f1
0,2,17.897858,0.0,0.0,0.0,0.0,0.0,0.012665,0.038897,0.013645,0.001170
1,4,17.897858,0.0,0.0,0.0,0.0,0.0,0.002641,0.039625,0.006787,0.005592
2,6,17.897858,0.0,0.0,0.0,0.0,0.0,0.001518,0.036024,0.004700,0.010413
3,8,17.897858,0.0,0.0,0.0,0.0,0.0,0.002848,0.034475,0.013793,0.003851
4,10,17.897858,0.0,0.0,0.0,0.0,0.0,0.004508,0.028414,0.010231,0.010947
5,12,17.897858,0.0,0.0,0.0,0.0,0.0,0.003853,0.024402,0.010304,0.009547


#### Save your results somewhere and load again to tabularize your results altogether

In [28]:
output_name = input("Plase give an output file name: ")

output_directory = f'../proxy_training_results/{model_path}/'
output_filename = os.path.join(output_directory, f'{output_name}.pkl')
print("Writing to file: ", output_filename)
with open(output_filename, 'wb') as f:
    pickle.dump(results, f)

Plase give an output file name: results
Writing to file:  ../proxy_training_results/BERT-fewshots-hdim-results/results.pkl
